In [1]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-09-16 20:44:58


# Key Package Components

In this section, each of the core modules will be discussed in greater detail.

---

## Core Module: `input_`

### `Geometry` class

This class is designed for parsing a user input string and returning a geometric data structure.  The raw *input string* is assumed to be a *geometry string* and is converted to Geometry object.  In summary:

    lamana.input_.Geometry(<input_str>) --> <Geometry object>

#### Geometry Data Structures

When an input string is formatted, it becomes a  *geometry string*.  An acceptible format is the **[General Convention](lpep.ipynb#LPEP-001:-Implementing-Coding-and-Package-Standards)** that represents characteristic laminae types of the format outer-[inner_i]-middle.

A `Geometry object` is combining mixed Pythonic types - specifically a `namedtuple` comprising floats, a list and a string (optional).  Summarize below:

|Object|Definition|Type|Example|
|:--|:--|:--:|:--|
|*input string*| a raw user input | `str` |`'400-200-800'` | 
|*geometry string*| formatted laminate geometry | `str` |`'400.0-[200.0]-800.0'`|
|*Geometry object*| `Geometry` class instance | `namedtuple` |`<Geometry object (400.0-[200.0]-800.0')>`|

By convention, names that reference geometry strings are typically lower-case, e.g.: 

- `g`, `geo_inputs`, `geos` or `geos_full` 
- `geos = ['400-200-800', '400-[100,100]-400S']`   

Names that reference `Geometry` objects are typically capatlized, e.g.: 

- `G`, `Geo_objects`, `Geos` or `Geos_full` 
- `G = la.input_.Geometry(FeatureInput)`

### `BaseDefaults` class

This class essentially stores common geometry strings and `Geometry` objects.  Placing them here allows simple inheritance of starter objects.  

There are two main dicts which are stored as instance attributes: `geo_inputs` and `Geo_objects`

####  The `geo_inputs` Dictionary

This dict comprises sample geometry strings where initial keys are named by the number of plies, e.g `'5-ply'`.  The number of total plies correlates to the total number of laminae types as determined by $$2(outer + inner) + middle$$  

Here is an example `geo_inputs` dict:

```
geo_inputs = {
    '1-ply': ['0-0-2000', '0-0-1000'],
    '2-ply': ['1000-0-0'],
    '3-ply': ['600-0-800', '600-0-400S'],
    '4-ply': ['500-500-0', '400-[200]-0'],
    '5-ply': ['400-200-800', '400-[200]-800', '400-200-400S'],
    '6-ply': ['400-[100,100]-0', '500-[250,250]-0'],
    '7-ply': ['400-[100,100]-800', '400-[100,100]-400S'],
    '9-ply': ['400-[100,100,100]-800'],
    '10-ply': ['500-[50,50,50,50]-0'],
    '11-ply': ['400-[100,100,100,100]-800'],
    '13-ply': ['400-[100,100,100,100,100]-800'],
}
```
Extra keys are added to this dict that dynamically append additional entries to this initial dict.  For instance, the keys 'geos_even', 'geos_odd' and 'geos_all' auto append sets of even, odd all geometry strings from the extant dictionary.  You can access the static and dynamic keys simply using the `.keys()` method.

Note the naming convention of using 's':

- `"geo_input**s**"`: the base dictionary
- `"geo**s**_<group>"`: sets of existing dict values appended to the dict.

Since this is a simiple `dict`, an author or developer can easily extend either the base or appended `dict` items.

#### The `Geo_objects` Dictionary

This is a lazy dictionary.  All entries of `geo_inputs` are automatically converted and stored as `Geometry` objects.  The purpose here is to eliminate the added step of calling Geometry to convert strings.  This `Geo_objects` and `geo_inputs` are both created using similar private methods, so there underlying mechanisms parallel.

#### Subclassing `BaseDefaults`

The remaining default objects such as `load_params`, `mat_props` and `FeatureInput` are particular to specific experimental setups and cannot be effectively generalized.  However, there is a `get_FeatureInput()` method use to help consistently format.  Additionally, this class can be subclassed to a custom `Defaults` class by the author, helpful in custom models.  This has a number of benefits for storing custom start values.  See the [Author Documentation](writecustom.ipynb#What-are-Defaults?) for examples of subclassing.

---

## Feature Module: `distributions`

### `Case` class

The `Case` class translates user information into managable, analytical units.  The are three main steps in using a `Case` object:

1. instantiate a `Case` instance
2. apply user information such as geometry strings, model name, etc.
3. access method and properties, such as `plot()` and `total`

Here is an idiomatic example of the latter characteristics:

    >>> case = la.distributions.Case(load_params, mat_props)
    >>> case.apply(geo_strings=None, model='Wilson_LT', **kwargs)
    >>> case.plot(**kwargs)

The `case` instance accepts loading and material information as dictionaries.  Specific geometry strings and a model are applied to the `case` instance.  The `apply()` method generates a `FeatureInput` object, which results in a `LaminateModel` object (or objects).   Information is parsed, calculated (such as layer thicknesses) and stored in attributes.  These attributes and methods are then accessible for performing analysis, most importantly the `plot()` method.

Therefore, you can think of a case as an analytical unit comprising start up data and a packaged `LaminateModel` objects.

### `Cases` class

The `Cases` class handles multiple case objects.  For example, set operations can be performed on multiple cases.  In this context, each `case` is termed a "`caselet`" that typically correlate with a matplotlib subplot.  Here is an idiomatic example:

    >>> import lamana as la
    
    >>> bdft = la.input_.BaseDefaults()
    >>> cases = Cases(bdft.geo_inputs['geos_all'], ps=[2,3,4])

The latter code uses defaults to build cases for all geometry strings contained in `BaseDefaults()`, one for each `p` number of datapoints.  In other words, in this example, *dozens* of analytical caselet units are built with only three lines of code.  See [LPEP 002](lpep.ipynb#LPEP-002:--Extending-Cases-with-Patterns) and [LPEP 003](lpep.ipynb#LPEP-003:--A-humble-case-for-caselets) for motivations and details regarding `Cases`. 

---

## Core Module: `constructs`

In principle, the `constructs` module builds a `LaminateModel` object.  Technically a `LaminateModel` is a `pandas` DataFrame representing a physical laminate with helpful attributes.  DataFrames provide a powerful data structure for visualizing and analyzing numerial data in a simple tabular format.

The `constructs` module computes laminate dimensional columns and compiles theoretical calculations handled by the complementary `theories` module.  Column names represent computational variables defined in the next sub-section.

### Preface: Variable Classifications

Before we discuss the laminate structure, we distinguish two ubiquitous variable categories used internally: `constructs` and `theories` variables.  In a full laminate DataFrame, these categories comprise columns of data are represented by columns.  The categories variables, columns and corresponding modules are illustrated in the image below and described in greater detail:

![dataframe output](./_images/dataframe_output.png)

This image shows the result data contained in a DataFrame.  Categories of columns (IDs, dimensionals and models).  The first two categories are computed by `constructs.Laminate` class.  This data is computed based on data supplied to FeatureInput.  The models columns are computed by the `theories` module and corresponding `models` classes.  The highlighted blue text indicates data supplied by user/author interaction.  For clarity, rows are colored with alternating red and orange colors to distinguish separate layers.  We quickly see the inherent visual benefits using a DataFrame.

- See the Appendix for clues on [distinguishing variable types in DataFrames](#constructs:-"Laminate"-vs.-"Model"-variables).
- See the Appendix for more [details on variables derived from models](#constructs:-More-on-Model-Variables).

### The `LaminateModel` Architecture

This section will describe in greater detail how `LaminateModel` objects are constructed.  

When the user calls `case.apply()`, a number of inherited objects are made.  The stages for building a `LaminateModel` object are outlined below and directly reflect the implementation of the `constructs` classes.

- Stage 1: build a `Stack`; a primitive laminate of order layers
- Stage 2: build a `Laminate`; calculate Laminate dimensional values (LFrame)
- Stage 3: build a `LaminateModel`; calculate laminate theory Model values (LMFrame)

`LaminateModel` inherits from `Laminate`, which inherits from the `Stack` class.
All accept a `FeatureInput` argument.


#### Stage 1: The `Stack` Class

The purpose of the `Stack` class is to build a skeletal, precusor of a primitive `Laminate` object.  This class houses methods for parsing `Geometry` objects into ordered layers, adding materials labels for each layer and setting *expected* stress states for each tensile or compressive side.  `Stack` returns a tuple containing stack-related information (described below).

For a given `Geometry` object instance the `Stack().StackTuple` method  creates a `namedtuple` conprising stack information.  `StackTuple` contains attributes to access:

- stack `order`
- the number of plies, `nplies`
- the technical `name` for the laminate, "4-ply", "5-ply"
- a convenient `alias` if any, e.g. "Bilayer", "Trilayer"

The `stack` attribute accesses a dictionary of the laminate layers ordered from bottom to top layers (tensile to compressive).  Although Python dictionaries are unsorted, this particular dictionary is ordered because each layer is enumerated and stored as keys to perserve the order, layer thickness and layer type (sometimes referred as "ltype").  

```python

Examples
--------
>>> import LamAna as la
>>> G = la.input_.Geometry(['400-200-800'])
>>> G
<Geometry object (400-[200]-800)>

Create a StackTuple and access its attributes
>>> st = constructs.Stack(G).StackTuple    # converts G to a namedtuple
>>> st.order                               # access namedtuple attributes
{1: [400.0, 'outer'],
 2: [200.0, 'inner']
 3: [800.0, 'middle']
 4: [200.0, 'inner']
 5: [400.0, 'outer']}
>>> st.nplies
5
>>> st.name
'5-ply'
>>> st.alias
'standard'

```

#### Stage 2: The `Laminate` class

The `Laminate` class prepares the geometric calculations of the laminate.  Given a `Stack` object, this class adds materials, stress sides and dimensional calculations.

`Geometry` information, material parameters and loading parameters are packed in a single  `FeatureInput` object - a dict of useful information that is passed between modules.  `Laminate` inherits from `Stack` and builds an LFrame object.   Stack information is stored in an instance attribute called `Snapshot` and then converted to a set of DataFrames.  

In summary, the IDs and dimensional data are determined and computed by `Stack` and `Laminate`.  Combined, this information builds an LFrame.


- See the Appendix for detail on [`FeatureInput`](#constructs:-More-on-FeatureInput).
- See the Appendix for details on [the `Snapshot` and other `Laminate` variables](#constructs:-More-on-Laminate-Variables).
- See the Appendix for details on [how materials are ordered in a stack](#constructs:-Material-Stacking-Order).

#### Stage 3: The `LaminateModel`

A `LaminateModel` object combines all dimensional information from `Laminate` with theoretical calculations produced from a laminate theory `Model`, e.g. stress/strain.  

Referring to the diagram, `LaminateModel._build_LMFrame()` calls `theories.handshake()` and tries to pass in an instance of itself.  The `self` at this point is a `Laminate` including an LFrame, which comprises IDs and Dimensional columns only.  An author, therefore, has full access to all `Laminate` attributes.  

From here, `theories.handshakes()` searches within the models directory for a model (grey, dashed arrows) specified by the user at the time of instantiation, i.e. `Case.apply(*args, model=<model_name>).` The resultant DataFrame is assigned to the `LaminateModel.LMFrame`) and a final `LaminateModel` is created.  Details of this workflow are illustrated in [`Laminate` Handling](#Core-Module:-theories).

### Data Structures

There are three main DataFrame types in LamAna.  Here we illustrate their similarities:

- `Snapshot`: a preview DataFrame of the `Stack` (see materials, layer info order); contains one row per layer.
- `LFrame`: updated `Snapshot` of IDs and dimensionals; contains `p` rows per layer.
- `LMFrame`: updated LFrame with models computed columns.

![laminate objects](./_images/laminate_objects.png)

The `LMFrame` is the primary data structure of interest containing all IDs, Dimensional and Model data and `p` number of rows pertaining to data points within a given lamina.  We will briefly summarize how to build the LMFrame and how columns are populated.  


#### ID Columns

The `Snapshot` updates the ID columns througe the `Laminate._build_snapshot()` method.  It is the precusor to the LFrame and the LMFrame.

    Variables addressed 
    -------------------
    `layer_, side_, matl_, type_, t_`

- See the Appendix for details on [the `Snapshot` variables](#constructs:-More-on-Laminate-Variables).

#### Dimensional Columns

Dimensional variable columns are populated through the `Laminate._build_LFrame()` method, which contains algorithms for calculating relative and absolute heights, thicknesses and midplane distances relative to the neutral axis.  These columns contain dimensional data that are determined independent from the laminate theory model. 

    Variables addresed 
    ------------------
    `label_, h_, d_, intf_, k_, Z_, z_`

These variables are defined in the `Laminate` class docstring.


- See the Appendix for [more details on the label variable](#constructs:-More-on-label_) to understand the role of points, `p` and their relationship to DataFrame rows.  

#### Model Columns

Finally, the Model variable columns are populated using `Laminate._build_LMFrame()`. These columns contain data based on calculations from laminate theory for a selected model.  Here global_vars and inline_vars are calculated.

    Variables addressed
    --------------------
    global_vars = [`v_eq, D_11T, D_12T, M_r, M_t, D_11p, D_12n, K_r, K_t`] --> FeatureInput['Global'] (dict entry)
    
    inline_vars = [`Q11, Q12, D11, D12, strain_r, strain_t, stress_r, stress_t, stress_f`] --> LaminateModel object (DataFrame)

- See the Appendix for more on [`global_vars` and `inline_vars`](#constructs:-More-on-Model-Variables).

---

## Core Module: `theories`

The module is responsbile for merging the `Laminate` date with theoretrical data  to produce a `LaminateModel`.

### `LaminateModel` Handling

For clarity, a diagram of `LaminateModel` handling is illustrated.  

The `Laminate` object, carrying the LFrame is passed from `constructs` to `theories`.  If successful, the `LaminateModel` is returned to `constructs`; otherwise an exception is caught in `constructs` and a `ModelError` is raised.  Further up in a Feature module, this error is handled and initiates a rollback to an LFrame.

![theories flowchart](./_images/diagram_theories.png)

A model is simply a module that contains code for handling laminate theory calculations.  The purpose of the model is to update the LFrame with Model variable columns of LT calculations.  `handshake()` automatically distinguishes whether the author implemented a class-style or function-style model.  The **most important hook method/function is `_use_model_()`**, which must be present somewhere inside the model module and must return a tuple containing:

    - an updated Laminate DataFrame with model data columns (a.k.a. `LaminateModel`)
    - a `FeatureInput` with updated `'Globals'` key.
    
`'Globals'` is a `dict` of calculated constants, used in exported reports [see output_ section](#Core-Module:-output_). 

Post-handshake, the self instance of the `LaminateModel` is updated with the new `LaminateModel` and `FeatureInput` (green arrow).  Otherwise exceptions are raised. A commom exception are for Laminates with `p=1`, which detects an `INDET` value in middle layers.  Handling these exceptions is done in the other modules.  

### Custom Models

Sometimes Classical Laminate Theory needs to be modified to fit a specific set of constraints or boundary conditions.  The LamAna package has powerful, extensible options for integrating user user-defined (authored) implementations of their own custom laminate theory models.  By convention, custom models are named by the author and suffixed by the characters "`_LT`"). 

A library of these custom models, tests and pre-defined defaults are stored in the `models` directory (sub-package).  Code for calculations, related exceptions, `FeatureInput` variables and defaults are stored in a models module.  The `theories` module then merge the model calculations with the procedures described above.

---

## Core Module: `output_`

A summary of `output` objects

| Object | Purpose |
|:------ |:-------- |
| `SinglePlot` | Stress distribution for a single geometry |
| `MultiPlot` | Stress distributions for a multiple geometries |
| `HalfPlot` | Partial plot of either compression or tension side |
| `QuarterPlot` | Partial halfplot excluding side without data |
| `PanelPlot` | A series of subplots side-by-side |
| `RatioPlot` | Ratio thickness plot; prinicipal stress vs. ratio |
| `PredictPlot` | Plot of experimental failure load or stress vs. middle layer princ. stress |

### Exporting Data

The `utils.tools.export()` function is used to save data as regular or temporary files of .xslx or .csv files.  Files are automatically stored in the default export folder. More details are shown in the Demonstrations file.

---

## Appendix

### `constructs`: "Laminate" vs. "Model" variables

- **Laminate** (or `constructs`) variables are responsible for building the laminate [stack](#First:-The-Stack-Class) and computing dimensional data.  Internally, these varibles will be semantically distinguished with a single trailing underscore.
    1. **ID**: variables related to layer and row identifications 
        1. `layer_`, `side_`, `matl_`, `type_`, `t_`
    2. **Dimensional**: variables of heights relative to cross-sectional planes
        1. `label_`, `h_`, `d_`, `intf_`, `k_`, `Z_`, `z_` 
- **Model** (or `theories`) variables are all remaining columns relevant for LT calculations as prescribed by a given model. Since these variables are author and model-specific, there is no unified semantic or naming convention.

The fine-grained details of model variables are not essential for typical API useage.  However, these detail may be helpful when authoring custom code that integrates with LamAna. Therefore, the next section with discuss this granuity, but may be overlooked by most readers.

### `constructs`: More on Model Variables

For more detailed discussions, model variables can be further divided into sub-categories.  There common subsets are as follows:

1. **User**: global variables delibrately set by the user at startup
2. **Inline**: variables used per lamina at a kth level (row)
3. **Global**: variables applied to the laminate, accessible by ks

Although model variables are often particular to a chosen model, e.g Wilson_LT, there are some  general trends that may be adopted.  Some model variables are provided at startup by the user (user_vars).  Some variables are calculated for each row in the data table (inline_vars).  Some variables are calculated by the designated laminate theory model, which provide constants for remaining calculations (global_vars).  Since global values are the same for every row, these constants are not included in the DataFrame, but they are stored internally within a Globals `dict`.  The details of this storage are coded within each model module.  

*Global* values are of particular importance to `FeatureInput` objects and when exporting meta data as dashboards in spreadsheets. In contrast, *Inline* values alter directly with the dimensional values thoroughout the lamainate thickness. Names of common variables used in `distributions` are organized below:

*Model Variable Subsets*

    Model_vars = {user_vars, inline_vars, global_vars}

*Examples of Subsets of Model Variables*

- user_vars   = [`mat_props`, `load_params`]
- global_vars = [`v_eq`, `D_11T`, `D_12T`, `M_r`, `M_t`, `D_11p`, `D_12n`, `K_r`, `K_t`]
- inline_vars = [`Q11`, `Q12`, `D11`, `D12`, `strain_r`, `strain_t`,
    `stress_r`, `stress_t`, `stress_f`]

Aside from user variables, all others are found as headers for columns in a DataFrame (or spreadsheet).

### `constructs`: More on `Laminate` Variables

The `Laminate._build_snapshot()`, extends the `Stack` object  by converting the ordered stack into DataFrame with assigned materials.  This `Snapshot` gives a preview of the laminate geometry, idenfiers (IDs) and stacking order materials. This "snapshot" has the following ID columns of infornation, which are accessible to the user in a `Case` instance (see `distributions.Case.snapshot`):

    Variables addressed: `layer_, matl_, type_, t_`

`Snapshot` is updated with labels of expected tensile and compressive stresses are located (`side_`) are assigned to a laminate through the `Laminate._set_stresses()` method.  This function accounts for DataFrames with even and odd rows.  For odd rows, 'None' is assigned to the neutral axis, implying "no stress".

    Variables addressed: `side_`
    
The final `Snapshot` serves as the "primitive" precursor to an LFrame.  In fact, `_build_primitive()` method takes a `snapshot` DataFrame and simply adds are `p` number of rows for each layer.  The resulting `_primitive` is essential for STAGE 2 of the `Laminate` construction.

### `constructs`: Material Stacking Order

The material order is initially defined by the user `mat_props` dict in `distributions` and automatically parsed in the `input_` module.  Extracting order from a dict is not trivial, so the default sorting is alphabetical order.  This order is handled by converting the dict to a pandas index.  See `Stack.add_materials()` method for more details.

As of 0.4.3d4, the user can partially override the default ordering by setting the `materials` property in the Case instance.  This allows simple control of the stacking order in the final laminate stack and `Laminate` objects. At the moment, a list of materials is cycled through; more customizations have not been implemented yet.

```python
>>> case.material
['HA', 'PSu']                                     # alphabetical order
>>> case.material = ['PSu', 'HA']                 # overriding order    
>>> case.material
['PSu', 'HA']
>>> case.apply(...)             
<materials DataFrame>                             # cycles the stacking order

```

### `constructs`: More on `FeatureInput`

A Feature module defines a `FeatureInput` object. 

For `distributions`, it is defined in `Case`. `FeatureInput`s contain information that is passed between objects.  For instance, this object transfers user input data in `distributions` (converted in `input_`) to the `constructs` module to build the laminate stack and populate ID and dimensional columns. A FeatureInput from `distributions` looks like the following (as of 0.4.4b).
```python
FeatureInput = {
    'Geometry': <Geometry object>,
    'Loading': <load_params dict>,
    'Materials': <mat_props dict>,
    'Custom': <undefined>,
    'Model': <string>,
    'Globals': <dict>,
}
```                    
After calculating model data, the "Globals" key is updated containing all necessary `globabl_vars`.  These variables are constant and are necessary for further calculations of `inline_vars`.  Here is an example of Global variables key-value pair in FeatureInput.

```python
FeatureInput['Globals'] = [v_eq, D_11T, D_12T, M_r, M_t, D_11p, D_12n, K_r, K_t]
```

### `constructs`: More on `label_`

See [LPEP 001.02](lpep.ipynb#LPEP-001:-Implementing-Coding-and-Package-Standards) for standards of API units.

For this explanation, imagine we transverse the absolute height of the laminate at different cross-sectional planes.  The values of inline stress points are calculated along different planes throughout the laminate thickness. What happens at interfaces where two materials meet with different stresses?  How are these two stress points differentiated in a DataFrame or in a plot?  For plotting purposes, we need to define different types of points in separate rows.  Here we define some rules and four types of points found within a (i.e. DataFrame rows):

1. interfacial - point on unbound outer surfaces and bound internal surfaces.
2. internal - point with the lamina thickness between interfaces 
3. discontinuity - point on bounded interfaces pertaining to an adjacent lamina
4. neutralaxis - the middle, symmetric axial plane

Of these points, only the interfacial point is determined by classical laminate theory.  How these points are distributed depends on their locations within each lamina and whether they are located on the tensile or compressive `side_`.   The image below illustrates the different points from above with respect to `k_` (the fractional height for a given layer).

![points](./_images/points.png)

The neutral axis exists in physical laminates, but they are only represented as a single row in a DataFrame of odd ply and odd `p`; they are not displayed in even laminates. 

Notice various layers have different point types:

- Middle layers have two interfacial points, no discontinuities and a neutral axis.
- If `p >= 2`, all other layers have one interfacial point with a discontinuity.
- Monoliths do not have discontinuities

### More on `IndeterminateError`

An `IndeterminateError` is thrown in when a value cannot be calculated.  An `INDET` value is substituted in appropriate DataFrame cells.  

An example for such an error is illustrated when determining the stress `side_` for a monolith with one data point (`nplies=1`, `p=1`).  From a design perspective, the location of the stress point is ambiguous, either on one interface or more intuitively at the center.  Of course, the center of the visual construct is at the neutral axis, which would report zero stress.  This position is misleading for representative stress state of the monolith.  Therefore, the `InderminateError` is thrown, recommending at least p = 2 for disambiguated stress calculations and reporting `INDET` for the indetermined value.  